In [2]:
# instalo la librería que me falta:
%pip install seaborn



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
# Importo las librerías que necesitaré para el sprint
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Utilizo 'connector' en lugar de Alchemy para conectarme a la BBDD:
import mysql.connector
conexion = mysql.connector.connect(user='root', password='SQLespe2010', host='localhost', database = 'transactions', port='3306')

# Verifico la conexión:
if conexion.is_connected():
    print(f"Conexión exitosa a la base de datos '{conexion.database}'")
else:
    print("Error al conectar")

# diccionario para almacenar totas las tablas de la BBDD mediante función:
def generar_diccionario_tablas(conexion):
    cursor = conexion.cursor()
    cursor.execute("SHOW TABLES;") 
    tablas = cursor.fetchall() 
    diccionario_tablas = {tabla[0]: f"SELECT * FROM {tabla[0]}" for tabla in tablas}
    return diccionario_tablas

# otro dicionario para almacenar los dataframes generados para cada tabla mediante función:
dataframes = {}
def cargar_tablas_en_dataframes(conexion):
    global dataframes
    query_dict = generar_diccionario_tablas(conexion)
    for tabla, query in query_dict.items():
        dataframes[tabla] = pd.read_sql(query, conexion)
        print(f"DataFrame creado para la tabla '{tabla}'")

cargar_tablas_en_dataframes(conexion)

print("\nNombres de los DataFrames creados:")
for df_name in dataframes:
    print(f"df_{df_name}")

#Cierro la conexión:
def cerrar_conexion():
    conexion.close()
    print("La conexión a la BBDD está cerrada")

cerrar = cerrar_conexion()

# visualizo la info de cada df:
for df_name, df in dataframes.items():
    print(f"\nInformación de '{df_name}':")
    df.info()  # Muestra información del DataFrame

Conexión exitosa a la base de datos 'transactions'
DataFrame creado para la tabla 'company'
DataFrame creado para la tabla 'credit_card'
DataFrame creado para la tabla 'data_user'
DataFrame creado para la tabla 'informetecnico'
DataFrame creado para la tabla 'transaction'
DataFrame creado para la tabla 'vistamarketing'

Nombres de los DataFrames creados:
df_company
df_credit_card
df_data_user
df_informetecnico
df_transaction
df_vistamarketing
La conexión a la BBDD está cerrada

Información de 'company':
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            101 non-null    object
 1   company_name  100 non-null    object
 2   phone         100 non-null    object
 3   email         100 non-null    object
 4   country       100 non-null    object
dtypes: object(5)
memory usage: 4.1+ KB

Información de 'credit_card':
<class 'pandas.core.fr

C:\Users\Espe\AppData\Local\Temp\ipykernel_9556\1827071490.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataframes[tabla] = pd.read_sql(query, conexion)


In [9]:
# EJERCICIO 1: una variable numèrica.
# tabla sales 

query = "SELECT amount FROM sales;"

df_sales = pd.read_sql(query, conexion)

# Gráfico el histograma
plt.figure(figsize=(8, 6))
sns.histplot(df_sales['amount'], kde=True, bins=20, color='blue')
plt.title("Distribución importe ventas")
plt.xlabel("Importe")
plt.ylabel("Frecuencia")
plt.grid(True)
plt.show()

C:\Users\Espe\AppData\Local\Temp\ipykernel_9556\1431252843.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sales = pd.read_sql(query, conexion)


DatabaseError: Execution failed on sql 'SELECT amount FROM sales;': 1146 (42S02): Table 'sprint4.sales' doesn't exist

In [ ]:
# EJERCICIO 2: Dues variables numèrica.

# Tabla = sales
# Variable = amount

query = "SELECT amount, quantity FROM sales;"

df_sales = pd.read_sql(query, conexion)


# Gráfico: scatter plot
plt.figure(figsize=(8, 6))
sns.scatterplot(data=df_sales, x='quantity', y='amount', color='green', alpha=0.7)
plt.title("Relación entre cantidad e importe ventas")
plt.xlabel("Cantidad")
plt.ylabel("Importe")
plt.grid(True)
plt.show()

In [1]:
# Ejercicio 3: Una variable categórica

# Tabla: products
# Variable: category

# Importo la tabla en pandas
query = "SELECT category FROM products;"
df_products = pd.read_sql(query, conexion)

# Creo variable para contar la frecuencia por categoría
contar_categoria = df_products['category'].value_counts()

# Gráfico: bar plot: muestra la frecuencia de cada categoría
plt.figure(figsize=(8, 6))
sns.barplot(x=contar_categoria.index, y=contar_categoria.values, palette='viridis')
plt.title("Frecuencia de categorías de productos")
plt.xlabel("Categoría")
plt.ylabel("Frecuencia")
plt.xticks(rotation=90)
plt.grid(True)
plt.show()

NameError: name 'pd' is not defined

In [ ]:
# EJERCICIO 4: Una variable categòrica i una numèrica.

# Gráfico: boxplot 
plt.figure(figsize=(8, 6))
sns.boxplot(data=df_sales_products, x='category', y='amount', palette='coolwarm')
plt.title("Distribución de importe por categoría")
plt.xlabel("Categoría")
plt.ylabel("Importe")
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

In [ ]:
# EJERCICIO 5: Dues variables categòriques

# Tabla: sales
# Variables: category y region

# Necesito la tabla de frecuencias previa
freq_table = pd.crosstab(df_sales_products['category'], df_sales_products['region'])

# Gráfico: heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(freq_table, annot=True, fmt="d", cmap='YlGnBu')
plt.title("Frecuencia conjunta entre categorías y regiones")
plt.xlabel("Región")
plt.ylabel("Categoría")
plt.show()

In [ ]:
# EJERCICIO 6: Tres variables.

# Tabla: sales
# Variables: amount, region, quantity


# Asigno colores y tamaños
region_sizes = df_sales['region'].value_counts()

# Graficar bubble chart
plt.figure(figsize=(10, 7))
sns.scatterplot(data=df_sales, x='quantity', y='amount', size='region', hue='region', sizes=(50, 500), palette='Set2')
plt.title("Relación entre cantidad-importe con regiones")
plt.xlabel("Cantidad")
plt.ylabel("Importe")
plt.legend(title="Región")
plt.grid(True)
plt.show()

In [ ]:
# EJERCICIO 7: Graficar un Pairplot.

#Tabla: sales.
#Variables: todas las numéricas (amount, quantity)

# Gráfico: pairplot
sns.pairplot(df_sales, diag_kind='kde', palette='husl')
plt.suptitle("Relaciones y distribuciones entre variables numéricas", y=1.03)
plt.show()

In [ ]:
conexion.commit()
conexion.close()